In [ ]:
# display the visual pattern that each filter is meant to respond to, apllying gradient ascent in the input space, maximizing the response of a specific filter and calculating the input images the filter is maximally responsive to
# they tell how a convolutional layer see the world, each layer see the input as a combination of these filters

In [ ]:
from keras import backend as K
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# customized metrics

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
model = load_model(os.path.join("..","models","CNN_dropout_batch_bigger_class_weights.h5"),custom_objects = {"precision": precision, "recall": recall})

In [ ]:
# turn it into a displayable image

def deprocess_image(x):
    # mean: 0, std: 1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clips to [0,1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# runs gradient ascent for a certain number of steps
steps = 40

def generate_pattern(layer_name, filter_index, size=69):
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])
    # gradient of the loss wrt to the model's input
    grads = K.gradients(loss, model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5) # normalize by dividing by its L2 norm
    iterate = K.function([model.input], [loss, grads])
    # starts from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.
    step = 1. # magnitude of each gradient update
    for i in range(steps):
        # obtain the loss value and gradient value
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step # adjust the input image in order to maximize the loss
    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
# pattern to which the i-th channel in layer is responsive
i = 7
layer = "conv2d_e"
plt.imshow(generate_pattern(layer, i))

In [ ]:
# grid of all filter response patterns in a layer

# for simplicity the first n filters of the layer set, obtaining a grid of the n filter patterns
layer_name = 'conv2d_2'
size = 69
margin = 5
results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3))
for i in range(8):
    for j in range(8):
        print(i, j)
        filter_img = generate_pattern(layer_name, i + (j * 8))
        horizontal_start = i * size + i * margin
        horizontal_end = horizontal_start + size
        vertical_start = j * size + j * margin
        vertical_end = vertical_start + size
        results[horizontal_start: horizontal_end,vertical_start: vertical_end, :] = filter_img
plt.figure(figsize=(20, 20))
plt.imshow(results)